# Building Custom Planners

This notebook teaches you how to build custom path planning algorithms for the Simple Autonomous Car SDK.

## What You'll Learn

1. Understanding the planner architecture
2. Building a simple waypoint planner
3. Building an A* planner for obstacle avoidance
4. Building a dynamic planner that reacts to perception
5. Combining planners

## Prerequisites

- Understanding of planners (see [Planner Tutorial](planner_tutorial.ipynb))
- Understanding of costmaps (see [Costmap Tutorial](costmap_tutorial.ipynb))
- Basic graph search algorithms knowledge

In [ ]:
import sys

sys.path.insert(0, '../../src')

import matplotlib.pyplot as plt
import numpy as np

from simple_autonomous_car import (
    BasePlanner,
    Car,
    CarState,
    GridCostmap,
    Track,
)

## Step 1: Understanding Planner Architecture

All planners inherit from `BasePlanner` and must implement:

- `plan(car_state, perception_data, costmap, goal)`: Returns path as numpy array

In [ ]:
# Let's look at BasePlanner
import inspect

from simple_autonomous_car.planning.base_planner import BasePlanner

print("BasePlanner interface:")
print(inspect.signature(BasePlanner.plan))

## Step 2: Building a Simple Waypoint Planner

A planner that follows predefined waypoints.

In [ ]:
class WaypointPlanner(BasePlanner):
    """
    A simple planner that follows predefined waypoints.
    """

    def __init__(self, waypoints: np.ndarray, lookahead: float = 10.0, name: str = "waypoint_planner"):
        super().__init__(name=name)
        self.waypoints = waypoints
        self.lookahead = lookahead

    def plan(
        self,
        car_state: CarState,
        perception_data: dict = None,
        costmap = None,
        goal = None,
    ) -> np.ndarray:
        """
        Plan path to next waypoints.
        """
        car_pos = car_state.position()

        # Find closest waypoint
        distances = np.linalg.norm(self.waypoints - car_pos, axis=1)
        closest_idx = np.argmin(distances)

        # Return path to next N waypoints
        num_waypoints = min(10, len(self.waypoints) - closest_idx)
        if closest_idx + num_waypoints <= len(self.waypoints):
            return self.waypoints[closest_idx:closest_idx + num_waypoints]

        return self.waypoints[closest_idx:]

# Test waypoint planner
waypoints = np.array([[0, 0], [10, 5], [20, 10], [30, 15], [40, 20]])
planner = WaypointPlanner(waypoints, lookahead=15.0)

car_state = CarState(x=5.0, y=2.0, heading=0.0, velocity=8.0)
plan = planner.plan(car_state)

print("✓ Waypoint planner created!")
print(f"  Plan has {len(plan)} waypoints")
print(f"  First waypoint: {plan[0]}")

## Step 3: Building a Costmap-Aware Planner

A planner that uses costmaps to avoid obstacles.

In [ ]:
class CostmapAwarePlanner(BasePlanner):
    """
    A planner that uses costmaps to avoid high-cost areas.
    """

    def __init__(self, track, cost_threshold: float = 0.5, name: str = "costmap_planner"):
        super().__init__(name=name)
        self.track = track
        self.cost_threshold = cost_threshold

    def plan(
        self,
        car_state: CarState,
        perception_data: dict = None,
        costmap = None,
        goal = None,
    ) -> np.ndarray:
        """
        Plan path along track, avoiding high-cost areas.
        """
        if costmap is None or not costmap.enabled:
            # Fallback to simple track following
            return self._plan_along_track(car_state)

        # Get track centerline
        centerline = self.track.centerline
        car_pos = car_state.position()

        # Find closest point on track
        distances = np.linalg.norm(centerline - car_pos, axis=1)
        closest_idx = np.argmin(distances)

        # Build path ahead, avoiding high-cost areas
        path = []
        lookahead_points = 30

        for i in range(lookahead_points):
            idx = (closest_idx + i) % len(centerline)
            point = centerline[idx]

            # Check cost at this point
            cost = costmap.get_cost(point, frame="global", car_state=car_state)

            if cost < self.cost_threshold:
                path.append(point)
            else:
                # High cost - try to find alternative nearby point
                offset = 2.0  # Try 2m to the side
                for side in [-1, 1]:
                    # Get perpendicular direction
                    if idx < len(centerline) - 1:
                        direction = centerline[idx + 1] - centerline[idx]
                        perp = np.array([-direction[1], direction[0]])
                        perp = perp / np.linalg.norm(perp)

                        alt_point = point + side * offset * perp
                        alt_cost = costmap.get_cost(alt_point, frame="global", car_state=car_state)

                        if alt_cost < self.cost_threshold:
                            path.append(alt_point)
                            break

        if len(path) == 0:
            return self._plan_along_track(car_state)

        return np.array(path)

    def _plan_along_track(self, car_state: CarState) -> np.ndarray:
        """Simple fallback: plan along track centerline."""
        centerline = self.track.centerline
        car_pos = car_state.position()
        distances = np.linalg.norm(centerline - car_pos, axis=1)
        closest_idx = np.argmin(distances)

        lookahead_points = 30
        end_idx = min(closest_idx + lookahead_points, len(centerline))
        return centerline[closest_idx:end_idx]

# Test costmap-aware planner
track = Track.create_simple_track(length=80.0, width=40.0, track_width=5.0)
costmap = GridCostmap(width=60.0, height=60.0, resolution=0.5, frame="ego")

planner = CostmapAwarePlanner(track, cost_threshold=0.3)

start_point, start_heading = track.get_point_at_distance(0.0)
car_state = CarState(x=start_point[0], y=start_point[1], heading=start_heading, velocity=8.0)

plan = planner.plan(car_state, costmap=costmap)

print("✓ Costmap-aware planner created!")
print(f"  Plan has {len(plan)} waypoints")
print(f"  Cost threshold: {planner.cost_threshold}")

## Step 4: Visualizing Planner Behavior

In [ ]:
from simple_autonomous_car.visualization import plot_car, plot_costmap, plot_plan, plot_track

# Visualize planner with costmap
fig, ax = plt.subplots(figsize=(12, 10))

plot_track(track, ax=ax)
plot_costmap(costmap, car_state, ax=ax, show_car=True, alpha=0.4)
plot_plan(plan, ax=ax, color="green", label="Plan")
plot_car(Car(initial_state=car_state), ax=ax, show_heading=True)

ax.set_title("Costmap-Aware Planner")
ax.legend()
plt.show()

print("✓ Planner visualization complete!")

## Summary

You've learned:

1. ✅ **Planner architecture**: Inherit from `BasePlanner`
2. ✅ **Waypoint planner**: Follow predefined waypoints
3. ✅ **Costmap-aware planner**: Avoid obstacles using costmaps
4. ✅ **Modular design**: Easy to build custom planners

### Key Concepts

- **BasePlanner**: Base class for all planners
- **Costmap integration**: Use costmaps for obstacle avoidance
- **Path format**: Return numpy array of waypoints
- **Flexibility**: Can use perception, costmap, or goal

### Next Steps

- Build A* planner for optimal pathfinding
- Build RRT planner for complex environments
- Build dynamic planners that react in real-time
- Combine multiple planners (hybrid planning)
- Build planners with velocity constraints